**TODO:**
    
- Where do checkpoints fit in?

I'm [building my own macOS Jupyter frontend](https://twitter.com/wasimlorgat/status/1611615676220817415?s=20) and writing about my experiences and learnings along the way. In order to do that, I need to be familiar with how Jupyter servers works.

This notebook is an executable playground for understanding and communicating with Jupyter servers. You can think of it as a barebones Jupyter frontend, since we'll be implementing the full lifecycle from creating a new notebook, writing code to it, and executing that code.

My approach to learning this was a combination of using Chrome dev tools to inspect network requests in Jupyter Lab, and reading the wonderful Jupyter server docs. I'll include links to the relevant docs in each section below.

Let's get started!

## Start the server

To start, ensure that you're running a Jupyter server in another process (e.g. in a terminal) by running the following command:

```sh
jupyter server
```

Then update the `url` and `token` variables below based on the terminal output. For example, it should output something like, but with a different `url` and `token`:

```
[C 2023-01-07 12:03:57.482 ServerApp]

    To access the server, open this file in a browser:
        file:///Users/seem/Library/Jupyter/runtime/jpserver-80287-open.html
    Or copy and paste one of these URLs:
        http://localhost:8890/?token=0799b0472813fd449b0ed5ee97431cc86b97045e88925063
     or http://127.0.0.1:8890/?token=0799b0472813fd449b0ed5ee97431cc86b97045e88925063
```

In [257]:
# NB: Update these based on your terminal output
url = 'http://localhost:8889/'
token = 'e751772ec65096e4d8984765e165de84717174f6dc3f3294'

## Authenticate

In [258]:
import requests

First, we'll do a quick check that there is a server at the defined `url`:

In [259]:
requests.get(url)

<Response [200]>

Next we need to authenticate. What happens if we try to make a request to an endpoint that requires authentication, for example `GET /api/sessions`?

In [260]:
requests.get(url + 'api/contents')

<Response [403]>

It fails with `403 Forbidden`.

If we include our token in the `Authorization` header:

In [261]:
headers = {'Authorization': f'token {token}'}
requests.get(url + 'api/contents', headers=headers)

<Response [200]>

... it works!

Let's create a `requests.Session` so we don't have to keep specifying headers:

In [262]:
session = requests.Session()
session.headers.update(headers)

## Managing files

### List the contents of a directory

In [263]:
!rm ../temp/*

`GET /api/contents/<path>` lists the contents of the directory at `path`. You can think of it as `ls`:

In [264]:
session.get(url + 'api/contents').json()

{'name': '',
 'path': '',
 'last_modified': '2023-01-12T08:00:46.849114Z',
 'created': '2023-01-12T08:00:46.849114Z',
 'content': [],
 'format': 'json',
 'mimetype': None,
 'size': None,
 'writable': True,
 'type': 'directory'}

Since the directory is currently empty, `content` is an empty list.

### Create an empty notebook

`POST /api/contents/<path>` creates an empty file in the directory at `path`. You can specify the `type` of the file in the request body:

In [265]:
session.post(url + 'api/contents', json={'type': 'notebook'})

<Response [201]>

Let's confirm that the file exists with `GET /api/contents`:

In [266]:
session.get(url + 'api/contents').json()

{'name': '',
 'path': '',
 'last_modified': '2023-01-12T08:00:49.132088Z',
 'created': '2023-01-12T08:00:49.132088Z',
 'content': [{'name': 'Untitled.ipynb',
   'path': 'Untitled.ipynb',
   'last_modified': '2023-01-12T08:00:49.133037Z',
   'created': '2023-01-12T08:00:49.133037Z',
   'content': None,
   'format': None,
   'mimetype': None,
   'size': 72,
   'writable': True,
   'type': 'notebook'}],
 'format': 'json',
 'mimetype': None,
 'size': None,
 'writable': True,
 'type': 'directory'}

The response is a nested dict. The root dict refers to the root directory as before, however, `content` now contains the newly created notebook file `Untitled.ipynb`.

We can get the contents of this file using the same method but referring to the file's path i.e. `GET /api/contents/<path>`:

In [267]:
data = session.get(url + 'api/contents/Untitled.ipynb').json()
data

{'name': 'Untitled.ipynb',
 'path': 'Untitled.ipynb',
 'last_modified': '2023-01-12T08:00:49.133037Z',
 'created': '2023-01-12T08:00:49.133037Z',
 'content': {'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

We're probably most interested in `content`, which contains the JSON content of the notebook:

In [268]:
data['content']

{'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5}

### Rename a notebook

Our newly created file is still named `Untitled.ipynb`. Let's rename it to `sum.ipynb` with `PATCH /api/contents/<path>`:

In [269]:
session.patch(url + 'api/contents/Untitled.ipynb', json={'path': 'sum.ipynb'}).json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-12T08:00:49.133037Z',
 'created': '2023-01-12T08:00:51.047123Z',
 'content': None,
 'format': None,
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

Confirm that it's been renamed. `Untitled.ipynb` no longer exists:

In [270]:
session.get(url + 'api/contents/Untitled.ipynb').json()

{'message': 'No such file or directory: Untitled.ipynb', 'reason': None}

... but `sum.ipynb` does:

In [271]:
session.get(url + 'api/contents/sum.ipynb').json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-12T08:00:49.133037Z',
 'created': '2023-01-12T08:00:51.047123Z',
 'content': {'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

::: {.callout-note}
You can also create a file with a specified name using `PUT /api/contents/<path>`, instead of letting the server find a unique named prefixed with `Untitled`.
:::

### Update a notebook's contents

Create a cell and append it to existing contents:

In [272]:
cell = {
    'cell_type': 'code',
    'id': '0',
    'metadata': {},
    'source': [
        '1 + 1\n',
    ],
    'outputs': [],
    'execution_count': 0,
}
data = session.get(url + 'api/contents/sum.ipynb').json()
data['content']['cells'].append(cell)

Update the notebook's contents using `PUT /api/contents/<path>`:

In [273]:
session.put(url + 'api/contents/sum.ipynb', json={'content': data['content'], 'type': 'notebook'})

<Response [200]>

Confirm that the notebook's been updated. Note that `last_modified` and `content` have both updated:

In [274]:
session.get(url + 'api/contents/sum.ipynb').json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-12T08:00:52.863452Z',
 'created': '2023-01-12T08:00:52.863452Z',
 'content': {'cells': [{'cell_type': 'code',
    'execution_count': 0,
    'id': '0',
    'metadata': {'trusted': True},
    'outputs': [],
    'source': '1 + 1\n'}],
  'metadata': {},
  'nbformat': 4,
  'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 216,
 'writable': True,
 'type': 'notebook'}

## Executing code

### Start a session

List open sessions with `GET /api/sessions`:

In [275]:
session.get(url + 'api/sessions').json()

[]

First we need to choose a kernel specification. Here are the available options:

In [276]:
session.get(url + 'api/kernelspecs').json()

{'default': 'python3',
 'kernelspecs': {'dyalog-kernel': {'name': 'dyalog-kernel',
   'spec': {'argv': ['python3',
     '-m',
     'dyalog_kernel',
     '-f',
     '{connection_file}'],
    'env': {},
    'display_name': 'Dyalog APL',
    'language': 'apl',
    'interrupt_mode': 'signal',
    'metadata': {}},
   'resources': {'kernel.js': '/kernelspecs/dyalog-kernel/kernel.js'}},
  'python3': {'name': 'python3',
   'spec': {'argv': ['python',
     '-m',
     'ipykernel_launcher',
     '-f',
     '{connection_file}'],
    'env': {},
    'display_name': 'Python 3 (ipykernel)',
    'language': 'python',
    'interrupt_mode': 'signal',
    'metadata': {'debugger': True}},
   'resources': {'logo-64x64': '/kernelspecs/python3/logo-64x64.png',
    'logo-32x32': '/kernelspecs/python3/logo-32x32.png',
    'logo-svg': '/kernelspecs/python3/logo-svg.svg'}}}}

Create a new session with `POST /api/sessions` with the `python3` kernelspec:

Note: Jupyter first creates a session with `path` as some UUID, then `GET`s the session, and patches the path.

Note: If a session already exists with the specified `name`, it'll be returned.

In [277]:
data = session.post(url + 'api/sessions', json={'kernel': {'name': 'python3'}, 'name': 'sum.ipynb', 'path': 'sum.ipynb', 'type': 'notebook'}).json()
data

{'id': 'c07a24b5-cea7-457c-a8eb-0bea14f84771',
 'path': 'sum.ipynb',
 'name': 'sum.ipynb',
 'type': 'notebook',
 'kernel': {'id': 'b01228ec-72a7-400b-a604-9d61ca134d01',
  'name': 'python3',
  'last_activity': '2023-01-12T08:01:00.310102Z',
  'execution_state': 'starting',
  'connections': 0},
 'notebook': {'path': 'sum.ipynb', 'name': 'sum.ipynb'}}

In [278]:
data = session.get(url + f'api/sessions/{data["id"]}').json()
data

{'id': 'c07a24b5-cea7-457c-a8eb-0bea14f84771',
 'path': 'sum.ipynb',
 'name': 'sum.ipynb',
 'type': 'notebook',
 'kernel': {'id': 'b01228ec-72a7-400b-a604-9d61ca134d01',
  'name': 'python3',
  'last_activity': '2023-01-12T08:01:00.310102Z',
  'execution_state': 'starting',
  'connections': 0},
 'notebook': {'path': 'sum.ipynb', 'name': 'sum.ipynb'}}

### Open a WebSocket

We'll use the [websockets](https://websockets.readthedocs.io/en/stable/) library.

In [280]:
%%writefile test.py
import asyncio
import datetime
import json
import websockets
import uuid

port = 8889
kernel_id = 'b01228ec-72a7-400b-a604-9d61ca134d01'
session_id = 'c07a24b5-cea7-457c-a8eb-0bea14f84771'
token = 'e751772ec65096e4d8984765e165de84717174f6dc3f3294'

code = '1 + 1'
code_msg_date = datetime.datetime.utcnow().isoformat()
code_msg_id = str(uuid.uuid1())
code_msg_d = {'buffers':[],'channel':'shell','content':{'silent':False,'store_history':True,'user_expressions':{},'allow_stdin':True,'stop_on_error':True,'code':code},
              'header':{'date':code_msg_date,'msg_id':code_msg_id,'msg_type':'execute_request','session':session_id,'username':'','version':'5.2'},
              'metadata':{'deletedCells':[],'recordTiming':False,'cellId':'0'},'parent_header':{}}
code_msg = json.dumps(code_msg_d)

def print_header(s, n=140):
    print('-'*n)
    print(s)
    print('-'*n)

def print_msg(msg):
    print(f"Received message      msg_type: {msg['msg_type']:16} content: {msg['content']}")

async def recv_json(websocket):
    return json.loads(await websocket.recv())

async def main():
    async with websockets.connect(f'ws://localhost:{port}/api/kernels/{kernel_id}/channels?session_id={session_id}', extra_headers={'Authorization': f'token {token}'}) as websocket:
        print_header('Receive initial messages')
        while True:
            try: msg = await asyncio.wait_for(recv_json(websocket), 1)
            except asyncio.TimeoutError: break
            print_msg(msg)
        
        print_header('Send execute_request')
        await websocket.send(code_msg)

        print_header('Receive execute_reply')
        while True:
            try: msg = await asyncio.wait_for(recv_json(websocket), 1)
            except TimeoutError: break
            print_msg(msg)
            if msg['msg_type'] == 'status' and msg['content']['execution_state'] == 'idle': break

asyncio.run(main())

Overwriting test.py


In [281]:
!python test.py

--------------------------------------------------------------------------------------------------------------------------------------------
Receive initial messages
--------------------------------------------------------------------------------------------------------------------------------------------
Received message      msg_type: status           content: {'execution_state': 'busy'}
Received message      msg_type: status           content: {'execution_state': 'idle'}
Received message      msg_type: status           content: {'execution_state': 'idle'}
--------------------------------------------------------------------------------------------------------------------------------------------
Send execute_request
--------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------

In [303]:
!rm test.py

## Cleanup

### Close the session

In [282]:
session.delete(url + f'api/sessions/{data["id"]}')

<Response [204]>

### Shutdown the server

In [283]:
session.post(url + 'api/shutdown')

<Response [200]>

In [302]:
try: session.get(url)
except requests.exceptions.ConnectionError: print('Server has been successfully shutdown!')

Server has been successfully shutdown!
